In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
with open("ccpp_parsed.json", "r") as f:
    result = json.load(f)


dfs_ebs = {}
for key in ["components", "connections"]:
    dfs_ebs[key] = pd.DataFrame.from_dict(result[key], orient="index").sort_index()

In [3]:
with open("ccpp_tespy.json", "r") as f:
    result = json.load(f)


dfs_tespy = {}
for key in ["components", "connections"]:
    dfs_tespy[key] = pd.DataFrame.from_dict(result[key], orient="index").sort_index()

In [4]:
dfs_ebs["connections"][["e_T", "e_M"]] = dfs_ebs["connections"][["e_T", "e_M"]].round(6)
dfs_tespy["connections"][["e_T", "e_M"]] = dfs_tespy["connections"][["e_T", "e_M"]].round(6)

In [5]:
columns = ["m", "p", "T", "e_T", "e_M"]

overlapping_index = list(set(dfs_tespy[key].index.tolist()) & set(dfs_ebs[key].index.tolist()))
diff_to_tespy = (
    (
        dfs_ebs[key].loc[overlapping_index, columns]
        - dfs_tespy[key].loc[overlapping_index, columns]
    ) / dfs_tespy[key].loc[overlapping_index, columns]
).abs().replace(np.inf, np.nan)
diff_to_ebs = (
    (
        dfs_ebs[key].loc[overlapping_index, columns]
        - dfs_tespy[key].loc[overlapping_index, columns]
    ).abs() / dfs_tespy[key].loc[overlapping_index, columns]
).abs().replace(np.inf, np.nan)

In [6]:
diff_to_ebs

,m,p,T,e_T,e_M
9,0.000133,0.000000e+00,8.699091e-05,6.961361e-05,0.000006
18,0.000051,0.000000e+00,1.080271e-09,1.470759e-07,0.000021
23,0.000133,1.705828e-10,1.982878e-11,2.753455e-08,0.000006
13,0.000212,0.000000e+00,7.429923e-16,3.213975e-05,0.000182
10,0.000250,0.000000e+00,9.279383e-05,5.955569e-05,0.000014
1,0.000025,0.000000e+00,0.000000e+00,NaN,NaN
22,0.000133,1.705828e-10,2.224224e-09,9.944618e-08,0.000006
21,0.000133,2.387121e-10,8.240234e-10,1.948957e-07,0.000006
15,0.000276,0.000000e+00,4.135872e-06,2.056249e-04,NaN
17,0.000212,0.000000e+00,3.154605e-11,8.531477e-06,0.000021


In [7]:
diff_to_ebs[diff_to_ebs > 1e-3].dropna(how="all")

,m,p,T,e_T,e_M
3,NaN,NaN,NaN,NaN,0.010478
2,NaN,NaN,0.001073,0.007147,0.001048
4,NaN,NaN,NaN,0.001259,0.001017
